In [4]:
import folium
from folium import CustomPane
import pandas as pd

# Sample DataFrame with "Lat", "Lng", "start_year", and "completion_year" columns
data = {
    "Lat": [51.5074, 48.8566, 40.7128, 45.4215],
    "Lng": [-0.1278, 2.3522, -74.0060, -75.6972],
    "start_year": [2000, 2010, None, 2030],  # None indicates no start year
    "completion_year": [2005, 2015, 2025, None]  # None indicates no completion year
}
df = pd.DataFrame(data)

# Create a Folium map centered around the mean of all coordinates
map_center = [df["Lat"].mean(), df["Lng"].mean()]
mymap = folium.Map(location=map_center, zoom_start=5)

# Function to update marker colors based on slider value
def update_markers(year):
    for layer in mymap._children.values():
        if isinstance(layer, folium.features.FeatureGroup):
            for feature in layer._children.values():
                if isinstance(feature, folium.map.Marker):
                    start_year = feature.options.get('data-start_year')
                    completion_year = feature.options.get('data-completion_year')
                    if completion_year and completion_year >= year:
                        if start_year and start_year >= year:
                            feature.options['icon'] = folium.Icon(color='red')
                        else:
                            feature.options['icon'] = folium.Icon(color='green')
                    else:
                        feature.options['icon'] = folium.Icon(color='gray')

# Create markers for each point in the DataFrame
for index, row in df.iterrows():
    completion_year = row['completion_year']
    if pd.notnull(completion_year):
        popup_text = f"Start Year: {row['start_year']}, Completion Year: {completion_year}"
        icon_color = 'gray'
        if row['start_year'] is not None:
            if row['start_year'] >= completion_year:
                icon_color = 'red'
            else:
                icon_color = 'green'
        folium.Marker(
            location=[row["Lat"], row["Lng"]],
            popup=popup_text,
            icon=folium.Icon(color=icon_color),
            tooltip=popup_text,
            data_start_year=row['start_year'],
            data_completion_year=completion_year
        ).add_to(mymap)

# Add slider as a CustomPane
slider_html = """
<div style="
    position: absolute;
    bottom: 10px;
    left: 50%;
    transform: translateX(-50%);
    z-index: 1000;
    background-color: rgba(255, 255, 255, 0.6);
    padding: 5px;
    border-radius: 5px;
    ">
    <input id="slider" type="range" min="2000" max="2030" value="2000" style="width: 300px;">
</div>
<script>
    var slider = document.getElementById('slider');
    slider.oninput = function() {
        window.postMessage({type: 'sliderInput', value: parseInt(this.value)});
    }
</script>
"""
pane = CustomPane(html=slider_html, z_index=1010)
mymap.get_root().add_child(pane)

# Function to handle slider value change
def handle_message(message):
    if message['type'] == 'sliderInput':
        update_markers(message['value'])

# Add listener to handle slider value change
mymap.add_child(folium.ElementHook(handle_message))

# Save the map as an HTML file
mymap.save("C:\\Users\\Lucas\\PycharmProjects\\LND\\coord_map.html")

ImportError: cannot import name 'CustomPane' from 'folium' (C:\Users\Lucas\anaconda3\Lib\site-packages\folium\__init__.py)